## Test accuracy on 20 CIFAR-100 classes using Xception

### Import all the necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import cifar100

2025-05-26 15:26:23.269069: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 15:26:23.477701: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 15:26:23.649528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748251583.799217   19285 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748251583.839689   19285 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748251584.185633   19285 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

###  Load CIFAR-100 fine labels (100-class)

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [3]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [4]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 71x71

In [5]:
x_train_20 = tf.image.resize(x_train_20, (71, 71)).numpy()
x_test_20 = tf.image.resize(x_test_20, (71, 71)).numpy()

E0000 00:00:1748251591.001945   19285 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1748251591.002563   19285 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Preprocess

In [6]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [7]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Use Xception + Custom Head

In [8]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(71, 71, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(20, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using Xception:", acc)

Epoch 1/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 53s 352ms/step - accuracy: 0.5339 - loss: 1.6045 - val_accuracy: 0.6800 - val_loss: 1.0476
Epoch 2/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 49s 345ms/step - accuracy: 0.7618 - loss: 0.7520 - val_accuracy: 0.7060 - val_loss: 0.9991
Epoch 3/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 49s 348ms/step - accuracy: 0.8240 - loss: 0.5550 - val_accuracy: 0.6970 - val_loss: 0.9774
Epoch 4/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 49s 346ms/step - accuracy: 0.8885 - loss: 0.3852 - val_accuracy: 0.7010 - val_loss: 1.0294
Epoch 5/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 48s 344ms/step - accuracy: 0.9219 - loss: 0.2745 - val_accuracy: 0.7060 - val_loss: 1.0746
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 159ms/step - accuracy: 0.7285 - loss: 0.9797
Test accuracy on 20 CIFAR-100 classes using Xception: 0.7245000004768372
